
MINE-4101: Ciencia de Datos Aplicada

Universidad de los Andes

Nombre: Juan Carlos Tovar
Código: 202216714

Fecha: Noviembre 22 - 2025



In [2]:
from scipy import stats
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [3]:
used_kmc_df = pd.read_excel("KMC-70k-93-2024-Obes 19-conVel-DATA-SPSS-20250322.xlsx")

In [4]:
used_kmc_df.sample(5)

,Idenfinal,Iden_Codigo,Iden_Sede,periodosanalisis,Sistemadeaseguramiento,Iden_embarazoMultiple,Iden_FechaParto,CSP_SituaPareja,CSP_TipoVivienda,CSP_EscolaridadMadre,...,formulaversusLME3m,mixtaversusLME3m,underweight12m,BMInacermas2DE,Nutpadre,velocidadzscorepeso40_3m,velocidadzscore3m_40semOMS,velocidad12_9mesesOMS,velocidad9_6mesesOMS,velocidad6_3mesesOMS
25119,1025389,25389,1,6,2,0,2022-08-22 00:00:00,3,2,3,...,#NULL!,#NULL!,#NULL!,0,1,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!
42173,2015432,15432,2,6,3,0,2021-12-30 00:00:00,1,2,5,...,0,0,0,0,1,-0.232365,-1.00758,0.307769,0.079946,-0.420802
11852,1011905,11905,1,5,2,0,2014-04-16 00:00:00,3,#NULL!,7,...,#NULL!,0,0,0,#NULL!,0.975629,0.259579,0.262111,-0.469604,-0.057686
44642,2017902,17902,2,#NULL!,3,0,2023-07-19 00:00:00,#NULL!,#NULL!,#NULL!,...,1,#NULL!,#NULL!,0,#NULL!,-0.564849,-1.145489,#NULL!,#NULL!,#NULL!
50957,3005891,5891,3,5,2,0,2017-05-20 00:00:00,3,2,7,...,#NULL!,0,0,0,0,1.608397,0.769785,0.417805,-0.139028,-0.675855


In [5]:
used_kmc_df.shape

(64801, 753)

In [6]:
used_kmc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64801 entries, 0 to 64800
Columns: 753 entries, Idenfinal to velocidad6_3mesesOMS
dtypes: int64(1), object(752)
memory usage: 372.3+ MB


In [7]:
# Muestra un listado de las columnas que tienen valores nulos (NaN)
used_kmc_df.isnull().sum()[used_kmc_df.isnull().sum() > 0].sort_values(ascending=False)

,0
V313,64758
V356,64756
V248,64738
V360C,64720
V423,64718
V402C,64710
V381,64705
V411,64701
V338,64681
V296,64663


In [8]:
# 1. Crear la serie de nulos a partir de used_kmc_df
nulos_series = used_kmc_df.isnull().sum()

# 2. Obtener el número total de filas del DataFrame original
total_filas = len(used_kmc_df)

# 3. Filtrar solo las columnas con nulos y ordenarlas
nulos_df = nulos_series[nulos_series > 0].sort_values(ascending=False).reset_index()

# 4. Renombrar las columnas para mayor claridad
nulos_df.columns = ['Columna', 'Conteo_Nulos']

# 5. Agregar la nueva columna con el total de datos (Total_Datos)
nulos_df['Total_Datos'] = total_filas

# Opcional: Agregar el porcentaje de nulos para una mejor visualización
nulos_df['Porcentaje_Nulos'] = (nulos_df['Conteo_Nulos'] / nulos_df['Total_Datos']) * 100

# 6. Mostrar las 20 columnas con más nulos
print("--- 20 Columnas con Mayor Cantidad de Nulos ---")
print(nulos_df.head(20))

--- 20 Columnas con Mayor Cantidad de Nulos ---
   Columna  Conteo_Nulos  Total_Datos  Porcentaje_Nulos
0     V313         64758        64801         99.933643
1     V356         64756        64801         99.930557
2     V248         64738        64801         99.902779
3    V360C         64720        64801         99.875002
4     V423         64718        64801         99.871916
5    V402C         64710        64801         99.859570
6     V381         64705        64801         99.851854
7     V411         64701        64801         99.845681
8     V338         64681        64801         99.814818
9     V296         64663        64801         99.787040
10    V369         64645        64801         99.759263
11    V376         64641        64801         99.753090
12    V418         64615        64801         99.712967
13    V333         64589        64801         99.672845
14   V317C         64580        64801         99.658956
15    V326         64542        64801         99.600315


In [9]:
# 1. Calcular el umbral de nulos (40% de las filas totales)
total_filas = len(used_kmc_df)
umbral_conteo = 0.40 * total_filas

# 2. Identificar las columnas a borrar
# Se usa used_kmc_df.isnull().sum() para obtener el conteo de nulos por columna
columnas_a_eliminar = used_kmc_df.columns[used_kmc_df.isnull().sum() > umbral_conteo]

# 3. Crear el nuevo DataFrame limpio
used_kmc_df_limpio = used_kmc_df.drop(columns=columnas_a_eliminar)

# 4. Mostrar el resumen de las operaciones
print("--- Resumen de Limpieza por Nulos (> 40%) ---")
print(f"Filas totales: {total_filas}")
print(f"Columnas originales: {used_kmc_df.shape[1]}")
print(f"Umbral de nulos (40%): {int(umbral_conteo)} nulos")
print(f"Columnas eliminadas: {len(columnas_a_eliminar)}")
print(f"Columnas finales: {used_kmc_df_limpio.shape[1]}")

--- Resumen de Limpieza por Nulos (> 40%) ---
Filas totales: 64801
Columnas originales: 753
Umbral de nulos (40%): 25920 nulos
Columnas eliminadas: 36
Columnas finales: 717


In [10]:
print("\n--- Listado de Columnas ELIMINADAS (Superan el 40% de Nulos) ---")
print(columnas_a_eliminar.tolist())


--- Listado de Columnas ELIMINADAS (Superan el 40% de Nulos) ---
['V203', 'V227', 'V236E', 'V245', 'V248', 'V254', 'V275C', 'V280', 'V284', 'V288', 'V291', 'V296', 'V301A', 'V313', 'V317C', 'V322', 'V326', 'V330', 'V333', 'V338', 'V344A', 'V356', 'V360C', 'V365', 'V369', 'V373', 'V376', 'V381', 'V386A', 'V402C', 'V407', 'V411', 'V415', 'V418', 'V423', 'V430A']


In [12]:
# 1. Definir el umbral de nulos (5% del total de filas)
total_filas = len(used_kmc_df_limpio)
umbral_5_porciento = 0.05 * total_filas

# 2. Identificar las columnas a imputar
# Buscamos columnas que tengan un conteo de nulos entre 1 y el 5% del total de filas
nulos_conteo = used_kmc_df_limpio.isnull().sum()
columnas_a_imputar = nulos_conteo[
    (nulos_conteo > 0) & (nulos_conteo < umbral_5_porciento)
].index.tolist()

if not columnas_a_imputar:
    print(" ¡No se encontraron columnas con nulos entre 0% y 5% en el DataFrame limpio. Ya estaban perfectas en ese rango!")
else:
    # 3. Separar las columnas identificadas por tipo de dato
    columnas_numericas = used_kmc_df_limpio[columnas_a_imputar].select_dtypes(include=np.number).columns.tolist()
    columnas_categoricas = used_kmc_df_limpio[columnas_a_imputar].select_dtypes(include=['object']).columns.tolist()

    # 4. Imputación de Columnas Numéricas con la MEDIANA
    # La mediana es ideal porque es resistente a los valores atípicos (outliers).
    for col in columnas_numericas:
        mediana_valor = used_kmc_df_limpio[col].median()
        used_kmc_df_limpio[col].fillna(mediana_valor, inplace=True)

    # 5. Imputación de Columnas Categóricas con la MODA
    # La moda (valor más frecuente) preserva la distribución de la categoría.
    for col in columnas_categoricas:
        # .mode()[0] obtiene el valor más frecuente
        moda_valor = used_kmc_df_limpio[col].mode()[0]
        used_kmc_df_limpio[col].fillna(moda_valor, inplace=True)

    print(f"--- Imputación Simple Completada ---")
    print(f"Total de columnas imputadas: {len(columnas_a_imputar)}")
    print(f"  - Numéricas (Mediana): {len(columnas_numericas)}")
    print(f"  - Categóricas (Moda): {len(columnas_categoricas)}")
    print("\n ¡La imputación simple se realizó con éxito! Todos los nulos en este rango han sido eliminados.")

    # Verificación final
    nulos_verificacion = used_kmc_df_limpio[columnas_a_imputar].isnull().sum().sum()
    print(f"Nulos restantes en estas columnas: {nulos_verificacion}")

 ¡No se encontraron columnas con nulos entre 0% y 5% en el DataFrame limpio. Ya estaban perfectas en ese rango!


In [13]:
# 1. Obtener el número total de filas
total_filas = len(used_kmc_df_limpio)

# 2. Definir los umbrales de conteo de nulos
umbral_bajo = 0.05 * total_filas  # 5%
umbral_alto = 0.40 * total_filas # 40%

# 3. Filtrar las columnas que caen en este rango
nulos_conteo = used_kmc_df_limpio.isnull().sum()

columnas_pendientes = nulos_conteo[
    (nulos_conteo >= umbral_bajo) & (nulos_conteo <= umbral_alto)
].sort_values(ascending=False)

if columnas_pendientes.empty:
    print(" ¡Felicidades! No quedan columnas con nulos entre el 5% y el 40%. El DataFrame está limpio, solo quedan las variables perfectas.")
else:
    print("--- Columnas Pendientes de Imputación (5% - 40% de Nulos) ---")
    print("Conteo | Porcentaje de Nulos")
    print("---------------------------------------")
    for nombre, conteo in columnas_pendientes.items():
        porcentaje = (conteo / total_filas) * 100
        print(f"{nombre}: {conteo} nulos ({porcentaje:.2f}%)")

 ¡Felicidades! No quedan columnas con nulos entre el 5% y el 40%. El DataFrame está limpio, solo quedan las variables perfectas.


In [14]:
# DataFrame limpio
df = used_kmc_df_limpio

# 1. Identificar todas las columnas que son de tipo 'object'
columnas_object = df.select_dtypes(include=['object']).columns

# Contador para seguimiento
columnas_convertidas = 0
nulos_creados = 0

print(f"--- Iniciando Conversión Forzada en {len(columnas_object)} columnas 'object' ---")

# 2. Iterar sobre las columnas 'object'
for col in columnas_object:
    # 3. Intentar la conversión a numérico.
    # errors='coerce' convierte cualquier texto no numérico a NaN.

    col_convertida = pd.to_numeric(df[col], errors='coerce')

    # 4. Verificar si la conversión fue exitosa y si vale la pena el cambio:
    if col_convertida.notna().sum() > (0.95 * len(df)):

        # Conteo de nulos creados
        nulos_antiguos = df[col].isnull().sum()
        nulos_nuevos = col_convertida.isnull().sum()

        if nulos_nuevos > nulos_antiguos:
            nulos_creados += (nulos_nuevos - nulos_antiguos)

        # 5. Aplicar la conversión al DataFrame original
        df[col] = col_convertida
        columnas_convertidas += 1

print(f"--- Conversión Terminada ---")
print(f"Columnas convertidas de 'object' a numérico: {columnas_convertidas}")
print(f"Nuevos nulos creados por caracteres inválidos: {nulos_creados}")

# Vuelve a verificar: ¿la conversión creó nuevos nulos?
print("\n--- Verificación de los 5 principales nulos ---")
print(df.isnull().sum().sort_values(ascending=False).head())

--- Iniciando Conversión Forzada en 716 columnas 'object' ---
--- Conversión Terminada ---
Columnas convertidas de 'object' a numérico: 72
Nuevos nulos creados por caracteres inválidos: 44215

--- Verificación de los 5 principales nulos ---
anoxia5mn               2566
SGAprema                2367
CSP_EscolaridadMadre    2007
CSP_SituaPareja         1874
CP_SA_InfUrinaria       1641
dtype: int64


In [15]:
# 1. Obtener el número total de filas
total_filas = len(used_kmc_df_limpio)

# 2. Calcular el conteo de nulos por columna (TODAS las columnas)
nulos_conteo = used_kmc_df_limpio.isnull().sum()

# 3. Filtrar solo las columnas que AÚN tengan nulos y ordenarlas
nulos_df_reporte = nulos_conteo[nulos_conteo > 0].sort_values(ascending=False).reset_index()

# 4. Asignar nombres a las columnas
nulos_df_reporte.columns = ['Variable', 'Conteo_Nulos']

# 5. Agregar las columnas de total y porcentaje
nulos_df_reporte['Total_Datos'] = total_filas
nulos_df_reporte['Porcentaje_Nulos'] = (nulos_df_reporte['Conteo_Nulos'] / nulos_df_reporte['Total_Datos']) * 100

print("--- REPORTE COMPLETO del Estado Actual de Nulos ---")
print("  (Ordenado de la variable con más nulos a la que menos tiene)")

if nulos_df_reporte.empty:
    print(" ¡El DataFrame está completamente limpio! No queda ninguna variable con nulos.")
else:
    # Mostrar todas las variables pendientes
    print(nulos_df_reporte)

--- REPORTE COMPLETO del Estado Actual de Nulos ---
  (Ordenado de la variable con más nulos a la que menos tiene)
                Variable  Conteo_Nulos  Total_Datos  Porcentaje_Nulos
0              anoxia5mn          2566        64801          3.959815
1               SGAprema          2367        64801          3.652721
2   CSP_EscolaridadMadre          2007        64801          3.097174
3        CSP_SituaPareja          1874        64801          2.891931
4        CP_SA_InfGineco          1641        64801          2.532368
..                   ...           ...          ...               ...
67             CONSULT12            16        64801          0.024691
68              filter_$            16        64801          0.024691
69           PrimaryLast            16        64801          0.024691
70               preterm            16        64801          0.024691
71                  LBWI            16        64801          0.024691

[72 rows x 4 columns]


In [17]:
print("--- Iniciando Imputación Final de Nulos (< 5%) ---")

# 1. Identificar todas las columnas que aún tienen nulos
columnas_con_nulos_final = used_kmc_df_limpio.columns[used_kmc_df_limpio.isnull().any()].tolist()

# 2. Separar por tipo de dato (Numéricas y Categóricas/Object)
nulos_numericos = used_kmc_df_limpio[columnas_con_nulos_final].select_dtypes(include=np.number).columns.tolist()
nulos_categoricos = used_kmc_df_limpio[columnas_con_nulos_final].select_dtypes(include=['object']).columns.tolist()

# 3. Imputación de Nulos Numéricos (MEDIANA)
if nulos_numericos:
    print(f"\nImputando {len(nulos_numericos)} Numéricas con la Mediana...")
    for col in nulos_numericos:
        mediana_valor = used_kmc_df_limpio[col].median()

        # BUENA PRÁCTICA: Asignar el resultado de fillna directamente a la columna
        used_kmc_df_limpio[col] = used_kmc_df_limpio[col].fillna(mediana_valor)

# 4. Imputación de Nulos Categóricos (MODA)
if nulos_categoricos:
    print(f"Imputando {len(nulos_categoricos)} Categóricas con la Moda...")
    for col in nulos_categoricos:
        moda_valor = used_kmc_df_limpio[col].mode()[0]

        # BUENA PRÁCTICA: Asignar el resultado de fillna directamente a la columna
        used_kmc_df_limpio[col] = used_kmc_df_limpio[col].fillna(moda_valor)

# 5. Verificación final de nulos
nulos_totales_restantes = used_kmc_df_limpio.isnull().sum().sum()

print("\n--- ¡Fase de Imputación Completada! ---")
print(f"Nulos totales restantes en todo el DataFrame: {nulos_totales_restantes}")
print("Advertencias de asignación eliminadas. ")

--- Iniciando Imputación Final de Nulos (< 5%) ---

--- ¡Fase de Imputación Completada! ---
Nulos totales restantes en todo el DataFrame: 0
Advertencias de asignación eliminadas. 


In [18]:
# 1. Identificar todas las columnas que son numéricas (int o float)
df_num = used_kmc_df_limpio.select_dtypes(include=np.number)
columnas_numericas = df_num.columns

outlier_reporte = {}
total_outliers = 0

print("--- Detección de Outliers (Método IQR) ---")

# 2. Iterar sobre cada columna numérica
for col in columnas_numericas:
    # 3. Calcular los cuartiles Q1 y Q3
    Q1 = df_num[col].quantile(0.25)
    Q3 = df_num[col].quantile(0.75)
    IQR = Q3 - Q1

    # 4. Definir los límites del outlier (1.5 * IQR)
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    # 5. Contar los valores que están fuera de los límites
    outliers_conteo = df_num[
        (df_num[col] < limite_inferior) | (df_num[col] > limite_superior)
    ].shape[0]

    if outliers_conteo > 0:
        outlier_reporte[col] = outliers_conteo
        total_outliers += outliers_conteo

# 6. Crear un DataFrame con el reporte y ordenar por conteo
outliers_df = pd.Series(outlier_reporte).sort_values(ascending=False).reset_index()
outliers_df.columns = ['Variable', 'Conteo_Outliers']

# 7. Agregar porcentaje
outliers_df['Porcentaje_Outliers'] = (outliers_df['Conteo_Outliers'] / len(used_kmc_df_limpio)) * 100

print(f"\nTotal de columnas numéricas revisadas: {len(columnas_numericas)}")
print(f"Columnas con Outliers: {len(outliers_df)}")
print(f"Conteo Total de Outliers: {total_outliers}")

print("\n--- Top 10 Columnas con Mayor Cantidad de Outliers ---")
print(outliers_df.head(10))

--- Detección de Outliers (Método IQR) ---

Total de columnas numéricas revisadas: 73
Columnas con Outliers: 53
Conteo Total de Outliers: 289171

--- Top 10 Columnas con Mayor Cantidad de Outliers ---
                 Variable  Conteo_Outliers  Porcentaje_Outliers
0  Sistemadeaseguramiento            23215            35.825064
1              edadmatcat            18019            27.806670
2           primariamadre            15513            23.939445
3                     BPN            15395            23.757349
4       oxigenoalaentrada            14703            22.689465
5                 toxemia            13667            21.090724
6               RCIUtalla            12694            19.589204
7               hospenURN            12026            18.558356
8      HD_DiasVenMecanica            11848            18.283668
9         CP_SA_InfGineco            11847            18.282125


In [19]:
# Lista de variables que tienen outliers altos y son probablemente códigos/categorías:
columnas_a_categorizar = [
    'Sistemadeaseguramiento', 'edadmatcat', 'primariamadre', 'BPN',
    'oxigenoalaentrada', 'toxemia', 'RCIUtalla', 'hospenURN', 'CP_SA_InfGineco'
    # Agrega aquí cualquier otra columna que sepas que es un código o booleana (0/1)
]

print("--- Iniciando Reclasificación de Códigos a Tipo 'category' ---")
for col in columnas_a_categorizar:
    used_kmc_df_limpio[col] = used_kmc_df_limpio[col].astype('category')
print(f"✅ {len(columnas_a_categorizar)} columnas convertidas a tipo 'category'.")

# Ahora tu DataFrame tiene menos columnas numéricas que generan falsos positivos.

--- Iniciando Reclasificación de Códigos a Tipo 'category' ---
✅ 9 columnas convertidas a tipo 'category'.


In [20]:
# Seleccionar solo las columnas numéricas reales
columnas_num_reales = used_kmc_df_limpio.select_dtypes(include=np.number).columns

print("\n--- Iniciando Capping (Tratamiento de Outliers) en Numéricas Reales ---")
outliers_tratados = 0

for col in columnas_num_reales:
    Q1 = used_kmc_df_limpio[col].quantile(0.25)
    Q3 = used_kmc_df_limpio[col].quantile(0.75)
    IQR = Q3 - Q1

    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    # Aplicar Capping: Reemplazar valores fuera de los límites
    num_cambios = used_kmc_df_limpio[col][
        (used_kmc_df_limpio[col] < limite_inferior) | (used_kmc_df_limpio[col] > limite_superior)
    ].count()

    used_kmc_df_limpio[col] = np.where(
        used_kmc_df_limpio[col] < limite_inferior, limite_inferior, used_kmc_df_limpio[col]
    )
    used_kmc_df_limpio[col] = np.where(
        used_kmc_df_limpio[col] > limite_superior, limite_superior, used_kmc_df_limpio[col]
    )
    outliers_tratados += num_cambios

print(f"✅ Tratamiento de Outliers (Capping) completado.")
print(f"Total de valores outliers reemplazados/tratados: {outliers_tratados}")


--- Iniciando Capping (Tratamiento de Outliers) en Numéricas Reales ---
✅ Tratamiento de Outliers (Capping) completado.
Total de valores outliers reemplazados/tratados: 152092


##Manejo de Variables Categóricas (Transformación para el Modelo)

In [21]:
import numpy as np
import pandas as pd

df = used_kmc_df_limpio

print("--- 🧐 Identificación de Variables Categóricas (object y category) ---")
print("-" * 70)
columnas_categoricas = []

for col in df.columns:
    # Verificamos si el tipo de dato es 'object' (cadenas) o 'category'
    if df[col].dtype == 'object' or df[col].dtype.name == 'category':
        unique_count = df[col].nunique()
        columnas_categoricas.append(col)

        # Impresión para ayudar a clasificar si es Nominal u Ordinal
        print(f"Columna: **{col}**")
        print(f"  Tipo de Dato: {df[col].dtype}")
        print(f"  Valores Únicos (Cardinalidad): {unique_count}")
        if unique_count <= 10:
            print(f"  Muestra de Categorías: {df[col].unique()}")
        elif unique_count < 50:
             print(f"  Muestra de Categorías: {df[col].unique()[:5]}... (y {unique_count - 5} más)")
        else:
            print("  ***ALTA CARDINALIDAD*** (Se recomienda revisar/descartar)")
        print("-" * 30)

if not columnas_categoricas:
    print("🎉 ¡No se encontraron columnas categóricas (object o category) para codificar!")

--- 🧐 Identificación de Variables Categóricas (object y category) ---
----------------------------------------------------------------------
Columna: **periodosanalisis**
  Tipo de Dato: object
  Valores Únicos (Cardinalidad): 7
  Muestra de Categorías: [2 3 4 5 '#NULL!' 6 1]
------------------------------
Columna: **Sistemadeaseguramiento**
  Tipo de Dato: category
  Valores Únicos (Cardinalidad): 3
  Muestra de Categorías: [3.0, 2.0, 0.0]
Categories (3, float64): [0.0, 2.0, 3.0]
------------------------------
Columna: **Iden_FechaParto**
  Tipo de Dato: object
  Valores Únicos (Cardinalidad): 5765
  ***ALTA CARDINALIDAD*** (Se recomienda revisar/descartar)
------------------------------
Columna: **CSP_TipoVivienda**
  Tipo de Dato: object
  Valores Únicos (Cardinalidad): 3
  Muestra de Categorías: ['#NULL!' 2 1]
------------------------------
Columna: **CSP_SituacionLaboralMadre**
  Tipo de Dato: object
  Valores Únicos (Cardinalidad): 5
  Muestra de Categorías: [2 '#NULL!' 1 3 4]
--

In [23]:
import numpy as np
import pandas as pd

# Lista de columnas de Alta Cardinalidad que deberían ser Numéricas
columnas_a_numerico = [
    'CSP_IngresoMensual', 'CP_PesoMadre', 'CP_TallaMadre', 'CP_tallamadremetro',
    'BMImadre', 'CP_PesoPadre', 'CP_TallaPadre', 'CP_tallapadremetro',
    'BMIpadre', 'HD_DiasVenNoInvasiva', 'HD_DiasCanulaNasa',
    'HD_DiasIncubadora', 'HD_DiasAlimentacionPar', 'HD_DiasUCI',
    'HD_DiasURN', 'HD_DiasAlojamiento', 'HD_PesoSalida',
    'AC_DiasPosCanguro', 'RELACIONAROSOBRETOTALCONSULTAS'
    # Agrega cualquier otra columna de Alta Cardinalidad que sea una medida/número
]

# 1. Reemplazar valores no numéricos ('#NULL!') por NaN de numpy
for col in columnas_a_numerico:
    df[col] = df[col].replace({'#NULL!': np.nan, ' ': np.nan}) # Limpia el valor y espacios en blanco

# 2. Forzar la conversión a tipo numérico (float)
for col in columnas_a_numerico:
    # errors='coerce' convierte cualquier cosa que no pueda ser float a NaN
    df[col] = pd.to_numeric(df[col], errors='coerce')

print(" Conversión a numérico de columnas clave completada. Se han generado nuevos NaNs.")

 Conversión a numérico de columnas clave completada. Se han generado nuevos NaNs.


In [24]:
# Imputación (rellenar NaNs):
for col in columnas_a_numerico:
    if df[col].isnull().sum() > 0:
        # Usamos la mediana para imputar, ya que es robusta a outliers
        mediana = df[col].median()
        df[col] = df[col].fillna(mediana)

print(" Imputación de los nuevos nulos (NaNs) por la mediana completada.")

 Imputación de los nuevos nulos (NaNs) por la mediana completada.


In [25]:
# Lista de columnas nominales de BAJA cardinalidad
columnas_nominales = [
    'periodosanalisis', 'Sistemadeaseguramiento', 'CSP_TipoVivienda',
    'CSP_SituacionLaboralMadre', 'CSP_SituacionLaboralPadre', 'CSP_DistanciaVivienda',
    'PA_LugarNacimiento', 'PA_AtendioParto', 'PA_TipoParto', 'PA_PresentacionNino',
    'ANOCAT2ISS', 'controlcat', 'pdpcat'
    # Incluye aquí todas las columnas de cardinalidad baja (2-10) que no son ordinales
]

# Codificación con One-Hot Encoding
df = pd.get_dummies(df, columns=columnas_nominales, prefix=columnas_nominales, drop_first=True)

print(" One-Hot Encoding para variables nominales completado.")

 One-Hot Encoding para variables nominales completado.


In [27]:
#import numpy as np
import pandas as pd

# 1. Definir la lista de columnas numéricas problemáticas
df = used_kmc_df_limpio

columnas_a_numerico = [
    'CSP_IngresoMensual', 'CP_PesoMadre', 'CP_TallaMadre', 'CP_tallamadremetro',
    'BMImadre', 'CP_PesoPadre', 'CP_TallaPadre', 'CP_tallapadremetro',
    'BMIpadre', 'HD_DiasVenNoInvasiva', 'HD_DiasCanulaNasa',
    'HD_DiasIncubadora', 'HD_DiasAlimentacionPar', 'HD_DiasUCI',
    'HD_DiasURN', 'HD_DiasAlojamiento', 'HD_PesoSalida',
    'AC_DiasPosCanguro', 'RELACIONAROSOBRETOTALCONSULTAS'
]

print("---  Análisis y Conversión de Columnas Numéricas Problemáticas ---")
print("-" * 60)
total_problemas = 0

# Crear una copia de los nulos antes de la conversión para comparar
nulos_antes = df[columnas_a_numerico].isnull().sum()

# 2. Reemplazar y forzar la conversión a numérico
for col in columnas_a_numerico:
    # 2a. Limpiar valores de texto conocidos por NaN (incluyendo el espacio en blanco y '#NULL!')
    df[col] = df[col].replace({'#NULL!': np.nan, ' ': np.nan})

    # 2b. Forzar la conversión a numérico (convierte cualquier otro error a NaN)
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 3. Calcular los nuevos nulos generados (los problemas reales)
nulos_despues = df[columnas_a_numerico].isnull().sum()
nulos_generados = nulos_despues - nulos_antes

# 4. Imprimir el resumen del impacto
for col in columnas_a_numerico:
    cambios = nulos_generados[col]
    if cambios > 0:
        print(f"Columna **{col}**: Se encontraron **{cambios}** valores problemáticos (no numéricos/NaNs).")
    total_problemas += cambios

print("-" * 60)
print(f"✨ **Total de valores problemáticos detectados y convertidos a NaN:** {total_problemas}")
print("-" * 60)


# 5. Imputación final de los nuevos nulos generados (usando la mediana)
print("\n--- 🩹 Imputación de NaNs generados (por la mediana) ---")
for col in columnas_a_numerico:
    if df[col].isnull().sum() > 0:
        mediana = df[col].median()
        df[col] = df[col].fillna(mediana)

print("✅ Imputación de las columnas numéricas problemáticas completada.")

---  Análisis y Conversión de Columnas Numéricas Problemáticas ---
------------------------------------------------------------
------------------------------------------------------------
✨ **Total de valores problemáticos detectados y convertidos a NaN:** 0
------------------------------------------------------------

--- 🩹 Imputación de NaNs generados (por la mediana) ---
✅ Imputación de las columnas numéricas problemáticas completada.


##Variables Ordinales

In [28]:
# 1. Mapeo de Escolaridad: El código original (1 a 8, más #NULL!)
mapeo_escolaridad = {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, '#NULL!': 0}
df['CSP_EscolaridadPadre_Cod'] = df['CSP_EscolaridadPadre'].map(mapeo_escolaridad)

# 2. Mapeo de Categorías de IMC (1, 2, 3, 4, más #NULL!)
mapeo_imc = {1: 1, 2: 2, 3: 3, 4: 4, '#NULL!': 0} # Asumiendo un orden de categoría (ej. 1=bajo, 4=alto)
df['BMImadrecat_Cod'] = df['BMImadrecat'].map(mapeo_imc)
df['BMIpadrecat_Cod'] = df['BMIpadrecat'].map(mapeo_imc)

# 3. Mapeo de Edad Materna Categórica (edadmatcat)
mapeo_edad = {1.0: 1, 2.0: 2, 3.0: 3} # Asumiendo 1.0, 2.0, 3.0 son ordenados
df['edadmatcat_Cod'] = df['edadmatcat'].map(mapeo_edad)

# Descartar las columnas ordinales originales de texto:
df = df.drop(columns=['CSP_EscolaridadPadre', 'BMImadrecat', 'BMIpadrecat', 'edadmatcat'], errors='ignore')

print("✅ Ordinal Encoding (Mapeo Manual) completado.")

/tmp/ipython-input-775668399.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['CSP_EscolaridadPadre_Cod'] = df['CSP_EscolaridadPadre'].map(mapeo_escolaridad)
/tmp/ipython-input-775668399.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['BMImadrecat_Cod'] = df['BMImadrecat'].map(mapeo_imc)
/tmp/ipython-input-775668399.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conc

✅ Ordinal Encoding (Mapeo Manual) completado.


In [29]:
import pandas as pd

# 1. Lista de columnas categóricas restantes a codificar (la lista grande)
columnas_finales_ohe = [
    'periodosanalisis', 'Sistemadeaseguramiento', 'CSP_TipoVivienda',
    'CSP_SituacionLaboralMadre', 'CSP_SituacionLaboralPadre', 'CSP_DistanciaVivienda',
    'CSP_numPersVivenIngMen', 'CSP_Menores5vivenMadre', 'CSP_NutricionFam',
    'CSP_EmbarazoDeseado', 'CSP_MesTabajoEmba', 'CSP_AyudaPerm1mes',
    'BMImadrecat', 'BMIpadrecat', 'CP_ARO', 'CP_EnfermeraJefe', 'CP_MedGeneral',
    'CP_Hormonales', 'CP_DIU', 'CP_MesInicCP', 'CP_SA_InfGineco', 'CP_SA_Anemia',
    'CP_SA_EnfRespiratoria', 'CP_SA_RPM', 'CP_SA_APP', 'CP_SA_Preclampsia',
    'CP_EstuvoHosp', 'CP_HospitalizacionesPreParto', 'CP_rhMadre',
    'CP_MadreAlcohol', 'CP_MadreDrogas', 'CP_MadreFumo',
    'CP_HospitalizacionEmbarazoAccidentes', 'CP_HospitalizacionEmbarazoAnemia',
    'CP_HospitalizacionEmbarazoEnfermedadRespiratoria', 'CP_HospitalizacionEmbarazoAPP',
    'CP_HospitalizacionEmbarazoPreeclampsia', 'CP_HospitalizacionEmbarazoEclampsia',
    'CP_HospitalizacionEmbarazoInfeccionGinecologica', 'CP_HospitalizacionEmbarazoInfeccionUrinaria',
    'CP_HospitalizacionEmbarazoSangrado', 'CP_TP_Orina', 'CP_TP_Rubeola',
    'CP_TP_Toxoplasmosis', 'CP_TP_Sifilis', 'CP_TP_HIV', 'CP_TP_HepatitisB',
    'CP_Medicamentos', 'CP_NumEcografias', 'CP_EcoPrimerTrimestre',
    'PA_LugarNacimiento', 'PA_AtendioParto', 'PA_TipoParto', 'PA_PresentacionNino',
    'PA_MovimientoDisminuido', 'PA_SufrimientoFetalAgudo', 'PA_LiqAmnioticoMeconiado',
    'PA_RitmoCardiaco', 'PA_Monitoreo', 'PA_NumDosisCorticoides',
    'PA_ComplicacionsPartoPlacentaPrevia', 'PA_ComplicacionsPartoAbrupcio',
    'PA_ComplicacionsPartoMultiples', 'PA_ComplicacionsPartoPatologIaCordOn',
    'PA_ComplicacionsPartoPreeclampsia', 'PA_ComplicacionsPartoEclampsia',
    'PA_ComplicacionsPartoAmnionitis', 'PA_ComplicacionsPartoSindromedehellp',
    'PA_DiasHospiMadre', 'PA_DiasUCIMadre', 'PA_EstActualMadre',
    'ERN_AdaptNeonatal', 'RCIUtalla', 'RCIUPC', 'zscorePC0cat',
    'ERN_A_1min', 'ERN_A_5min', 'ERN_A_10min', 'ERN_Remision',
    'HD_DiasFototerapia', 'HD_DiasCamaraCefalica', 'HD_ExamenesEcografiacerebral',
    'HD_ExamenesEcocardiograma', 'HD_ExamenesExamendeorina', 'HD_ExamenesRubeola',
    'HD_ExamenesToxoplasmosis', 'HD_ExamenesSifilis', 'HD_ExamenesHIV',
    'HD_ExamenesHepatitisB', 'HD_ExamenesGenetico', 'HD_ExamenesRMICerebral',
    'HD_ExamenesTSH_Tiroides', 'HD_DosisSurfactante', 'HD_CicloAntibio',
    'HD_DosisAmino', 'HD_CiclosUltimaLinea', 'HD_CiclosOtroAntibio', 'HD_RhNino',
    'HD_NumTrasSanguineas', 'HD_C_HemorragiaIntra', 'HD_C_Hipoglicemia',
    'HD_C_Apnea', 'HD_C_Ictericia', 'HD_C_ExamenNeurologico', 'HD_C_Convulsiones',
    'HD_C_DisplasiaBronco', 'HD_C_OxigenoDependencia', 'HD_TipoAlimentacionS',
    'AC_LugarAC', 'AC_MaxHorasLogroCargar', 'AC_AdapSeno', 'AC_ExtraSeno',
    'AC_AdmGoteo', 'AC_UnidadAbierta', 'AC_MadreAmamanta', 'AC_CansaNinoChupando',
    'AC_EntrenoOtroCuidador', 'AC_RecibioChupo', 'AC_CirujiaSeno',
    'AC_SoportePosicionC', 'AC_ApoyoLactancia', 'AC_EnsePosCangu', 'AC_SillaIncubadora',
    'V10D', 'V10B', 'V10C', 'ANOCAT2ISS', 'educpadresimplificada',
    'ERN_sepsis', 'HD_C_GradoHIV', 'Sufrimientofetalcronico', 'tuvotransfusiones',
    'toxemia', 'ecocat', 'apgarcat1', 'apgarcat5', 'ninosmenosde5anos', 'UCI',
    'ALIMENTAPARENTERAL', 'INFECCIONOSOCOMIAL', 'BPN', 'Nearterm', 'controlcat',
    'pdpcat', 'primipara', 'trimestre', 'anemiamadre', 'HD_Infecciones',
    'recibioantibioticos', 'AROCAT02', 'tipoventilacion'
]

# 2. Filtrar la lista: Solo incluir las columnas que existen en el DataFrame
columnas_existentes = [col for col in columnas_finales_ohe if col in df.columns]

# 3. Aplicar One-Hot Encoding SIN el argumento 'errors'
df = pd.get_dummies(
    df,
    columns=columnas_existentes,
    prefix=columnas_existentes,
    dummy_na=True,
    drop_first=True
)

print("✅ One-Hot Encoding final completado con la versión de Pandas compatible.")

✅ One-Hot Encoding final completado con la versión de Pandas compatible.


## Creación variable objetivo 'Riesgo_Neurocognitivo_Anual

In [30]:
import numpy as np
import pandas as pd

# Nombres de columna confirmados para los desenlaces a 12 meses (Edad Corregida)
COLUMNA_IQ = 'CD12'
COLUMNA_INFANIB = 'infanib12m'

# Aseguramos que la columna IQ sea numérica, convirtiendo errores a NaN.
df[COLUMNA_IQ] = pd.to_numeric(df[COLUMNA_IQ], errors='coerce')

# --- INICIO DE LA LÓGICA CORREGIDA ---

# 1. Definir la condición de riesgo de INFANIB
# Usamos una lista de valores explícitos que indican riesgo/anormalidad y SOLO los aplicamos si el valor NO ES NaN.
# Si el valor original es NaN, df[COLUMNA_INFANIB].notna() será False y no se incluirá aquí.

# Valores que pueden indicar riesgo o anormalidad en INFANIB
valores_riesgo_infanib = ['ANORMAL', 'SOSPECHOSO', 'RIESGO']
valores_riesgo_infanib = [v.upper() for v in valores_riesgo_infanib] # Convertir a mayúsculas para consistencia

# Crear la Serie de INFANIB en mayúsculas, llenando NaNs con un valor temporal (ej., 'NA')
infanib_upper = df[COLUMNA_INFANIB].astype(str).str.upper().fillna('NO_APLICA')

# Condición 1: El valor de INFANIB está explícitamente en la lista de riesgo
condicion_infanib_anormal = infanib_upper.isin(valores_riesgo_infanib)

# 2. Definir la condición de riesgo de IQ
condicion_iq_bajo = df[COLUMNA_IQ] < 85

# 3. Aplicar la lógica OR (Disyunción Lógica)
df['Riesgo_Neurocognitivo_Anual'] = np.where(
    condicion_infanib_anormal | condicion_iq_bajo,
    1,  # Riesgo Positivo (1)
    0   # Normal (0)
)

# 4. Mostrar el resumen
print(f"✅ Variable objetivo 'Riesgo_Neurocognitivo_Anual' CORREGIDA.")
print("-" * 60)
print(f"   Casos de Riesgo (1): {df['Riesgo_Neurocognitivo_Anual'].sum()} registros")
print(f"   Casos Normales (0): {len(df) - df['Riesgo_Neurocognitivo_Anual'].sum()} registros")

✅ Variable objetivo 'Riesgo_Neurocognitivo_Anual' CORREGIDA.
------------------------------------------------------------
   Casos de Riesgo (1): 3207 registros
   Casos Normales (0): 61594 registros


/tmp/ipython-input-738604839.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Riesgo_Neurocognitivo_Anual'] = np.where(


In [31]:
# Suponemos que X_filtered ya fue definida por el Paso 1
if 'X_filtered' in locals() or 'X_filtered' in globals():

    # Excluimos la columna 'const' (el intercepto) ya que no es un predictor.
    variables_restantes = X_filtered.columns.drop('const', errors='ignore').tolist()

    num_columnas = len(variables_restantes)

    print(f"--- ✅ Listado COMPLETO de Variables Predictoras Restantes para el Modelo 1 ---")
    print(f"Total de Variables sin Varianza Cero: {num_columnas}\n")

    # Imprimimos el listado completo
    for var in variables_restantes:
        print(f"* {var}")

    print("\n--- Fin del listado ---")

else:
    print("❌ Error: La variable 'X_filtered' no está definida en la memoria.")
    print("Por favor, ejecuta el bloque de código del Paso 1 (Definir la Matriz Limpia) de nuevo para cargarla.")

❌ Error: La variable 'X_filtered' no está definida en la memoria.
Por favor, ejecuta el bloque de código del Paso 1 (Definir la Matriz Limpia) de nuevo para cargarla.


## Eliminación varianza cero





In [32]:
import statsmodels.api as sm
import numpy as np
import pandas as pd

# --- 1. PREPARACIÓN Y ELIMINACIÓN DE VARIANZA CERO ---

COLUMNAS_Y_EXCLUIR = ['CD12', 'infanib12m', 'Riesgo_Neurocognitivo_Anual']
PATRONES_EXCLUIR = ['40sem', '3mEC', '6mEC', '9mEC', '12m', '12mEC', 'PCANG_', 'SEG_',
                    'AC_MaxHorasLogroCargar', 'DiasTot', 'REHOSP', 'CONSULT', 'HD_DiasUCI',
                    'HD_DiasURN', 'HD_PesoSalida', 'NM_']

X_cols_inicial = [
    col for col in df.columns
    if not any(pat in col for pat in PATRONES_EXCLUIR) and col not in COLUMNAS_Y_EXCLUIR
]
Y = df['Riesgo_Neurocognitivo_Anual']
X = df[X_cols_inicial].copy()

# Limpieza y Conversión a Numérico
X.replace(['#NULL!', '', ' ', 'nan'], np.nan, inplace=True)
X = X.replace([np.inf, -np.inf], np.nan)
X = X.apply(pd.to_numeric, errors='coerce')
X = X.dropna(axis=1, how='any')

# Añadir la constante
X = sm.add_constant(X, has_constant='add')
X = X.astype(float)

# ELIMINACIÓN DE VARIANZA CERO
most_common_value_ratio = X.apply(lambda col: col.value_counts(normalize=True).iloc[0] if not col.empty else 0)
cols_to_drop_variance = most_common_value_ratio[most_common_value_ratio > 0.995].index.tolist()

if 'const' in cols_to_drop_variance:
    cols_to_drop_variance.remove('const')

print(f"Eliminando {len(cols_to_drop_variance)} columnas por tener varianza casi cero (frecuencia > 99.5%).")
X_filtered = X.drop(columns=cols_to_drop_variance, errors='ignore')
print("✅ Variable 'X_filtered' definida y lista.")

/tmp/ipython-input-2011353615.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X.replace(['#NULL!', '', ' ', 'nan'], np.nan, inplace=True)


Eliminando 508 columnas por tener varianza casi cero (frecuencia > 99.5%).
✅ Variable 'X_filtered' definida y lista.


##Variables candidatas al modelo

In [33]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# --- 1. DEFINICIÓN DE LA MATRIZ INICIAL (Basado en el Modelo 1) ---

COLUMNAS_Y_EXCLUIR = ['CD12', 'infanib12m', 'Riesgo_Neurocognitivo_Anual']
# Excluimos datos de seguimiento posteriores para enfocarnos en el Modelo 1 (al nacimiento)
PATRONES_EXCLUIR_MODELO_1 = ['3mEC', '6mEC', '9mEC', '12m', '12mEC', 'PCANG_', 'SEG_', 'REHOSP', 'CONSULT', 'NM_']

X_cols_inicial = [
    col for col in df.columns
    if not any(pat in col for pat in PATRONES_EXCLUIR_MODELO_1) and col not in COLUMNAS_Y_EXCLUIR
]
X = df[X_cols_inicial].copy()

# 2. LIMPIEZA RIGUROSA (Nulos y Varianza Cero)

X.replace(['#NULL!', '', ' ', 'nan'], np.nan, inplace=True)
X = X.replace([np.inf, -np.inf], np.nan)
X = X.apply(pd.to_numeric, errors='coerce')
X = X.dropna(axis=1, how='any') # Eliminación por datos faltantes

X_const = sm.add_constant(X, has_constant='add')

# Eliminación de Varianza Casi Cero (Frecuencia > 99.5%)
most_common_value_ratio = X_const.apply(lambda col: col.value_counts(normalize=True).iloc[0] if not col.empty else 0)
cols_to_drop_variance = most_common_value_ratio[most_common_value_ratio > 0.995].index.tolist()
if 'const' in cols_to_drop_variance:
    cols_to_drop_variance.remove('const')

X_no_const = X_const.drop(columns=cols_to_drop_variance, errors='ignore').drop(columns='const', errors='ignore')

# --- 3. LIMPIEZA VIF EXTREMA (Definición de X_vif_filtered_final) ---

X_vif_input = X_no_const.copy()

def calculate_vif_single_pass(df):
    vif_data = pd.DataFrame()
    vif_data["feature"] = df.columns
    # El cálculo es rápido, pero puede mostrar RuntimeWarning por VIF infinito
    vif_data["VIF"] = [variance_inflation_factor(df.values, i) for i in range(len(df.columns))]
    return vif_data

vif_threshold_extremo = 1000.0

print("Realizando limpieza VIF extrema para definir la matriz final...")
try:
    vif_results = calculate_vif_single_pass(X_vif_input)
    cols_to_remove = vif_results[vif_results['VIF'] >= vif_threshold_extremo]['feature'].tolist()

    # DEFINICIÓN DE LA VARIABLE FINAL
    X_vif_filtered_final = X_vif_input.drop(columns=cols_to_remove, errors='ignore')
    print(f"✅ Matriz X_vif_filtered_final definida. Eliminadas {len(cols_to_remove)} columnas por multicolinealidad extrema.")

except Exception as e:
    print(f"❌ Error durante el cálculo VIF: {e}. Usando X_no_const completa.")
    X_vif_filtered_final = X_vif_input.copy()

# --- 4. EXTRACCIÓN DEL LISTADO FINAL ---

variables_finales = X_vif_filtered_final.columns.tolist()

print("\n--- ✅ Lista FINAL de Variables Candidatas del Modelo 1 ---")
print(f"Total de Variables Viables (Modelo al Nacimiento): {len(variables_finales)}\n")

for i, var in enumerate(variables_finales):
    print(f"[{i+1}] {var}")

/tmp/ipython-input-3850963447.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X.replace(['#NULL!', '', ' ', 'nan'], np.nan, inplace=True)


Realizando limpieza VIF extrema para definir la matriz final...
❌ Error durante el cálculo VIF: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''. Usando X_no_const completa.

--- ✅ Lista FINAL de Variables Candidatas del Modelo 1 ---
Total de Variables Viables (Modelo al Nacimiento): 547

[1] Idenfinal
[2] Iden_Codigo
[3] Iden_Sede
[4] Iden_FechaParto
[5] CSP_SituaPareja
[6] CSP_EscolaridadMadre
[7] CSP_IngresoMensual
[8] CP_PesoMadre
[9] CP_TallaMadre
[10] CP_tallamadremetro
[11] BMImadre
[12] CP_PesoPadre
[13] CP_TallaPadre
[14] CP_tallapadremetro
[15] BMIpadre
[16] CP_TotalCPN
[17] CP_SA_Sangrado
[18] CP_SA_InfUrinaria
[19] CP_edadmaterna
[20] ERN_Peso
[21] ERN_Talla0
[22] ERN_Talla
[23] ERN_Sexo
[24] RCIUpeso
[25] pesotalla0
[26] ERN_FUM
[27] ERN_Ballard
[28] zscorepesotalla0
[29] zscorepeso0
[30] zscoretalla0
[31] ERN_LubchencoFenton
[32] HD_FechaEntrada
[33] HD_FechaSalida


## Division de los datos (train-test)

In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score



# 1. Definición de las variables predictoras (X) y objetivo (Y)
# Usaremos los nombres de las 547 variables de tu lista.
variables_candidatas = [
    'Idenfinal', 'Iden_Codigo', 'Iden_Sede', 'Iden_FechaParto', 'CSP_SituaPareja', 'CSP_EscolaridadMadre',
    'CSP_IngresoMensual', 'CP_PesoMadre', 'CP_TallaMadre', 'CP_tallamadremetro', 'BMImadre', 'CP_PesoPadre',
    'CP_TallaPadre', 'CP_tallapadremetro', 'BMIpadre', 'CP_TotalCPN', 'CP_SA_Sangrado', 'CP_SA_InfUrinaria',
    'CP_edadmaterna', 'ERN_Peso', 'ERN_Talla0', 'ERN_Talla', 'ERN_Sexo', 'RCIUpeso', 'pesotalla0', 'ERN_FUM',
    'ERN_Ballard', 'zscorepesotalla0', 'zscorepeso0', 'zscoretalla0', 'ERN_LubchencoFenton', 'HD_FechaEntrada',
    'HD_FechaSalida', 'HD_DiasVenNoInvasiva', 'HD_DiasOxigeno', 'HD_DiasCanulaNasa', 'HD_DiasIncubadora',
    'HD_DiasAlimentacionPar', 'HD_DiasUCI', 'HD_DiasURN', 'HD_DiasAlojamiento', 'HD_TotalDiasHospital',
    'HD_PesoSalida', 'AC_DiasPosCanguro', 'V7a', 'V7', 'V10A', 'VAR00003', 'ANOCAT', 'educmadresimplificada',
    'pesocat', 'gestacat', 'cesarea', 'V195B', 'anoxia5mn', 'V195', 'VAR00005', 'VAR00004', 'V196', 'V196D',
    'V196A', 'EGEnt', 'EG1', 'V215', 'V259', 'V302', 'V387', 'oxigenoalaentrada', 'hospenURN', 'gestasal',
    'preterm', 'gananciapesonacerpesoentradaPMC', 'primariamadre', 'SGAprema', 'CSP_EscolaridadPadre_Cod',
    'BMImadrecat_Cod', 'BMIpadrecat_Cod', 'edadmatcat_Cod', 'periodosanalisis_2', 'periodosanalisis_3',
    'periodosanalisis_4', 'periodosanalisis_5', 'periodosanalisis_6', 'periodosanalisis_#NULL!',
    'Sistemadeaseguramiento_2.0', 'Sistemadeaseguramiento_3.0', 'CSP_TipoVivienda_2', 'CSP_TipoVivienda_#NULL!',
    'CSP_SituacionLaboralMadre_2', 'CSP_SituacionLaboralMadre_3', 'CSP_SituacionLaboralMadre_4',
    'CSP_SituacionLaboralMadre_#NULL!', 'CSP_SituacionLaboralPadre_2', 'CSP_SituacionLaboralPadre_3',
    'CSP_SituacionLaboralPadre_#NULL!', 'CSP_DistanciaVivienda_2', 'CSP_DistanciaVivienda_3',
    'CSP_DistanciaVivienda_4', 'CSP_DistanciaVivienda_#NULL!', 'CSP_numPersVivenIngMen_2',
    'CSP_numPersVivenIngMen_3', 'CSP_numPersVivenIngMen_4', 'CSP_numPersVivenIngMen_5',
    'CSP_numPersVivenIngMen_6', 'CSP_numPersVivenIngMen_7', 'CSP_numPersVivenIngMen_8',
    'CSP_numPersVivenIngMen_#NULL!', 'CSP_Menores5vivenMadre_1', 'CSP_Menores5vivenMadre_2',
    'CSP_Menores5vivenMadre_3', 'CSP_Menores5vivenMadre_#NULL!', 'CSP_NutricionFam_2', 'CSP_NutricionFam_3',
    'CSP_NutricionFam_#NULL!', 'CSP_EmbarazoDeseado_1', 'CSP_EmbarazoDeseado_#NULL!', 'CSP_MesTabajoEmba_1',
    'CSP_MesTabajoEmba_2', 'CSP_MesTabajoEmba_3', 'CSP_MesTabajoEmba_4', 'CSP_MesTabajoEmba_5',
    'CSP_MesTabajoEmba_6', 'CSP_MesTabajoEmba_7', 'CSP_MesTabajoEmba_8', 'CSP_MesTabajoEmba_9',
    'CSP_MesTabajoEmba_#NULL!', 'CSP_AyudaPerm1mes_1', 'CSP_AyudaPerm1mes_#NULL!', 'CP_ARO_1', 'CP_ARO_2',
    'CP_ARO_3', 'CP_ARO_4', 'CP_ARO_5', 'CP_ARO_6', 'CP_ARO_7', 'CP_ARO_8', 'CP_ARO_9', 'CP_ARO_10',
    'CP_ARO_12', 'CP_ARO_15', 'CP_ARO_20', 'CP_ARO_#NULL!', 'CP_EnfermeraJefe_1', 'CP_EnfermeraJefe_2',
    'CP_EnfermeraJefe_3', 'CP_EnfermeraJefe_4', 'CP_EnfermeraJefe_5', 'CP_EnfermeraJefe_6',
    'CP_EnfermeraJefe_7', 'CP_EnfermeraJefe_8', 'CP_EnfermeraJefe_#NULL!', 'CP_MedGeneral_1',
    'CP_MedGeneral_2', 'CP_MedGeneral_3', 'CP_MedGeneral_4', 'CP_MedGeneral_5', 'CP_MedGeneral_6',
    'CP_MedGeneral_7', 'CP_MedGeneral_8', 'CP_MedGeneral_9', 'CP_MedGeneral_#NULL!', 'CP_Hormonales_1',
    'CP_Hormonales_#NULL!', 'CP_DIU_1', 'CP_DIU_#NULL!', 'CP_MesInicCP_1', 'CP_MesInicCP_2', 'CP_MesInicCP_3',
    'CP_MesInicCP_4', 'CP_MesInicCP_5', 'CP_MesInicCP_6', 'CP_MesInicCP_7', 'CP_MesInicCP_#NULL!',
    'CP_SA_InfGineco_1.0', 'CP_SA_Anemia_1', 'CP_SA_Anemia_#NULL!', 'CP_SA_EnfRespiratoria_1',
    'CP_SA_EnfRespiratoria_#NULL!', 'CP_SA_RPM_1', 'CP_SA_RPM_#NULL!', 'CP_SA_APP_1', 'CP_SA_APP_#NULL!',
    'CP_SA_Preclampsia_1', 'CP_SA_Preclampsia_#NULL!', 'CP_EstuvoHosp_1', 'CP_EstuvoHosp_#NULL!',
    'CP_HospitalizacionesPreParto_1', 'CP_HospitalizacionesPreParto_#NULL!', 'CP_rhMadre_2', 'CP_rhMadre_4',
    'CP_rhMadre_5', 'CP_rhMadre_6', 'CP_rhMadre_7', 'CP_rhMadre_8', 'CP_rhMadre_#NULL!', 'CP_MadreAlcohol_1',
    'CP_MadreAlcohol_#NULL!', 'CP_MadreDrogas_1', 'CP_MadreDrogas_#NULL!', 'CP_MadreFumo_1',
    'CP_MadreFumo_#NULL!', 'CP_HospitalizacionEmbarazoAccidentes_#NULL!',
    'CP_HospitalizacionEmbarazoAnemia_#NULL!', 'CP_HospitalizacionEmbarazoEnfermedadRespiratoria_#NULL!',
    'CP_HospitalizacionEmbarazoAPP_1', 'CP_HospitalizacionEmbarazoAPP_#NULL!',
    'CP_HospitalizacionEmbarazoPreeclampsia_1', 'CP_HospitalizacionEmbarazoPreeclampsia_#NULL!',
    'CP_HospitalizacionEmbarazoEclampsia_#NULL!', 'CP_HospitalizacionEmbarazoInfeccionGinecologica_1',
    'CP_HospitalizacionEmbarazoInfeccionGinecologica_#NULL!', 'CP_HospitalizacionEmbarazoInfeccionUrinaria_1',
    'CP_HospitalizacionEmbarazoInfeccionUrinaria_#NULL!', 'CP_HospitalizacionEmbarazoSangrado_1',
    'CP_HospitalizacionEmbarazoSangrado_#NULL!', 'CP_TP_Orina_1', 'CP_TP_Orina_2', 'CP_TP_Orina_3',
    'CP_TP_Orina_4', 'CP_TP_Orina_#NULL!', 'CP_TP_Rubeola_3', 'CP_TP_Rubeola_#NULL!', 'CP_TP_Toxoplasmosis_2',
    'CP_TP_Toxoplasmosis_3', 'CP_TP_Toxoplasmosis_4', 'CP_TP_Toxoplasmosis_#NULL!', 'CP_TP_Sifilis_2',
    'CP_TP_Sifilis_4', 'CP_TP_Sifilis_#NULL!', 'CP_TP_HIV_4', 'CP_TP_HIV_#NULL!', 'CP_TP_HepatitisB_4',
    'CP_TP_HepatitisB_#NULL!', 'CP_Medicamentos_1', 'CP_Medicamentos_2', 'CP_Medicamentos_3',
    'CP_Medicamentos_4', 'CP_Medicamentos_5', 'CP_Medicamentos_99', 'CP_Medicamentos_#NULL!',
    'CP_NumEcografias_0', 'CP_NumEcografias_1', 'CP_NumEcografias_2', 'CP_NumEcografias_3',
    'CP_NumEcografias_4', 'CP_NumEcografias_5', 'CP_NumEcografias_6', 'CP_NumEcografias_7',
    'CP_NumEcografias_8', 'CP_NumEcografias_9', 'CP_NumEcografias_10', 'CP_NumEcografias_12',
    'CP_NumEcografias_15', 'CP_NumEcografias_20', 'CP_NumEcografias_#NULL!', 'CP_EcoPrimerTrimestre_1',
    'CP_EcoPrimerTrimestre_#NULL!', 'PA_LugarNacimiento_#NULL!', 'PA_AtendioParto_#NULL!', 'PA_TipoParto_2',
    'PA_TipoParto_3', 'PA_TipoParto_#NULL!', 'PA_PresentacionNino_2', 'PA_PresentacionNino_3',
    'PA_PresentacionNino_#NULL!', 'PA_MovimientoDisminuido_1', 'PA_MovimientoDisminuido_#NULL!',
    'PA_SufrimientoFetalAgudo_1', 'PA_SufrimientoFetalAgudo_#NULL!', 'PA_LiqAmnioticoMeconiado_1',
    'PA_LiqAmnioticoMeconiado_#NULL!', 'PA_RitmoCardiaco_1', 'PA_RitmoCardiaco_#NULL!', 'PA_Monitoreo_1',
    'PA_Monitoreo_#NULL!', 'PA_NumDosisCorticoides_1', 'PA_NumDosisCorticoides_2', 'PA_NumDosisCorticoides_3',
    'PA_NumDosisCorticoides_#NULL!', 'PA_ComplicacionsPartoPlacentaPrevia_1',
    'PA_ComplicacionsPartoPlacentaPrevia_#NULL!', 'PA_ComplicacionsPartoAbrupcio_#NULL!',
    'PA_ComplicacionsPartoMultiples_1', 'PA_ComplicacionsPartoMultiples_#NULL!',
    'PA_ComplicacionsPartoPatologIaCordOn_#NULL!', 'PA_ComplicacionsPartoPreeclampsia_1',
    'PA_ComplicacionsPartoPreeclampsia_#NULL!', 'PA_ComplicacionsPartoEclampsia_#NULL!',
    'PA_ComplicacionsPartoAmnionitis_#NULL!', 'PA_ComplicacionsPartoSindromedehellp_#NULL!',
    'PA_DiasHospiMadre_1', 'PA_DiasHospiMadre_2', 'PA_DiasHospiMadre_3', 'PA_DiasHospiMadre_4',
    'PA_DiasHospiMadre_5', 'PA_DiasHospiMadre_6', 'PA_DiasHospiMadre_7', 'PA_DiasHospiMadre_8',
    'PA_DiasHospiMadre_#NULL!', 'PA_DiasUCIMadre_1', 'PA_DiasUCIMadre_2', 'PA_DiasUCIMadre_3',
    'PA_DiasUCIMadre_4', 'PA_DiasUCIMadre_#NULL!', 'PA_EstActualMadre_3', 'PA_EstActualMadre_#NULL!',
    'ERN_AdaptNeonatal_2', 'ERN_AdaptNeonatal_3', 'ERN_AdaptNeonatal_#NULL!', 'RCIUtalla_1.0', 'RCIUPC_1',
    'RCIUPC_#NULL!', 'zscorePC0cat_2', 'zscorePC0cat_3', 'zscorePC0cat_4', 'zscorePC0cat_#NULL!',
    'ERN_A_1min_2', 'ERN_A_1min_3', 'ERN_A_1min_4', 'ERN_A_1min_5', 'ERN_A_1min_6', 'ERN_A_1min_7',
    'ERN_A_1min_8', 'ERN_A_1min_9', 'ERN_A_1min_#NULL!', 'ERN_A_5min_5', 'ERN_A_5min_6', 'ERN_A_5min_7',
    'ERN_A_5min_8', 'ERN_A_5min_9', 'ERN_A_5min_10', 'ERN_A_5min_#NULL!', 'ERN_A_10min_7', 'ERN_A_10min_8',
    'ERN_A_10min_9', 'ERN_A_10min_10', 'ERN_A_10min_#NULL!', 'ERN_Remision_1', 'ERN_Remision_#NULL!',
    'HD_DiasFototerapia_1', 'HD_DiasFototerapia_2', 'HD_DiasFototerapia_3', 'HD_DiasFototerapia_4',
    'HD_DiasFototerapia_5', 'HD_DiasFototerapia_#NULL!', 'HD_DiasCamaraCefalica_1',
    'HD_DiasCamaraCefalica_#NULL!', 'HD_ExamenesEcografiacerebral_1', 'HD_ExamenesEcografiacerebral_#NULL!',
    'HD_ExamenesEcocardiograma_1', 'HD_ExamenesEcocardiograma_2', 'HD_ExamenesEcocardiograma_#NULL!',
    'HD_ExamenesExamendeorina_1', 'HD_ExamenesExamendeorina_#NULL!', 'HD_ExamenesRubeola_#NULL!',
    'HD_ExamenesToxoplasmosis_1', 'HD_ExamenesToxoplasmosis_#NULL!', 'HD_ExamenesSifilis_#NULL!',
    'HD_ExamenesHIV_#NULL!', 'HD_ExamenesHepatitisB_#NULL!', 'HD_ExamenesGenetico_#NULL!',
    'HD_ExamenesRMICerebral_#NULL!', 'HD_ExamenesTSH_Tiroides_1', 'HD_ExamenesTSH_Tiroides_#NULL!',
    'HD_DosisSurfactante_1', 'HD_DosisSurfactante_2', 'HD_DosisSurfactante_3', 'HD_DosisSurfactante_#NULL!',
    'HD_CicloAntibio_1', 'HD_CicloAntibio_2', 'HD_CicloAntibio_#NULL!', 'HD_DosisAmino_1', 'HD_DosisAmino_#NULL!',
    'HD_CiclosUltimaLinea_1', 'HD_CiclosUltimaLinea_#NULL!', 'HD_CiclosOtroAntibio_1',
    'HD_CiclosOtroAntibio_2', 'HD_CiclosOtroAntibio_3', 'HD_CiclosOtroAntibio_#NULL!', 'HD_RhNino_2',
    'HD_RhNino_4', 'HD_RhNino_5', 'HD_RhNino_6', 'HD_RhNino_7', 'HD_RhNino_8', 'HD_RhNino_#NULL!',
    'HD_NumTrasSanguineas_1', 'HD_NumTrasSanguineas_2', 'HD_NumTrasSanguineas_3',
    'HD_NumTrasSanguineas_#NULL!', 'HD_C_HemorragiaIntra_1', 'HD_C_HemorragiaIntra_#NULL!',
    'HD_C_Hipoglicemia_1', 'HD_C_Hipoglicemia_#NULL!', 'HD_C_Apnea_1', 'HD_C_Apnea_2', 'HD_C_Apnea_#NULL!',
    'HD_C_Ictericia_1', 'HD_C_Ictericia_2', 'HD_C_Ictericia_3', 'HD_C_Ictericia_#NULL!',
    'HD_C_ExamenNeurologico_1', 'HD_C_ExamenNeurologico_2', 'HD_C_ExamenNeurologico_3',
    'HD_C_ExamenNeurologico_4', 'HD_C_ExamenNeurologico_#NULL!', 'HD_C_Convulsiones_#NULL!',
    'HD_C_DisplasiaBronco_1', 'HD_C_DisplasiaBronco_#NULL!', 'HD_C_OxigenoDependencia_1',
    'HD_C_OxigenoDependencia_#NULL!', 'HD_TipoAlimentacionS_2', 'HD_TipoAlimentacionS_3',
    'HD_TipoAlimentacionS_#NULL!', 'AC_LugarAC_2', 'AC_LugarAC_3', 'AC_LugarAC_5', 'AC_LugarAC_#NULL!',
    'AC_MaxHorasLogroCargar_1', 'AC_MaxHorasLogroCargar_2', 'AC_MaxHorasLogroCargar_3',
    'AC_MaxHorasLogroCargar_4', 'AC_MaxHorasLogroCargar_5', 'AC_MaxHorasLogroCargar_6',
    'AC_MaxHorasLogroCargar_7', 'AC_MaxHorasLogroCargar_8', 'AC_MaxHorasLogroCargar_10',
    'AC_MaxHorasLogroCargar_12', 'AC_MaxHorasLogroCargar_20', 'AC_MaxHorasLogroCargar_24',
    'AC_MaxHorasLogroCargar_#NULL!', 'AC_AdapSeno_2', 'AC_AdapSeno_3', 'AC_AdapSeno_#NULL!',
    'AC_ExtraSeno_2', 'AC_ExtraSeno_3', 'AC_ExtraSeno_#NULL!', 'AC_AdmGoteo_2', 'AC_AdmGoteo_3',
    'AC_AdmGoteo_#NULL!', 'AC_UnidadAbierta_1', 'AC_UnidadAbierta_2', 'AC_UnidadAbierta_3',
    'AC_UnidadAbierta_#NULL!', 'AC_MadreAmamanta_1', 'AC_MadreAmamanta_#NULL!', 'AC_CansaNinoChupando_1',
    'AC_CansaNinoChupando_#NULL!', 'AC_EntrenoOtroCuidador_1', 'AC_EntrenoOtroCuidador_#NULL!',
    'AC_RecibioChupo_1', 'AC_RecibioChupo_#NULL!', 'AC_CirujiaSeno_1', 'AC_CirujiaSeno_#NULL!',
    'AC_SoportePosicionC_1', 'AC_SoportePosicionC_#NULL!', 'AC_ApoyoLactancia_1', 'AC_ApoyoLactancia_#NULL!',
    'AC_EnsePosCangu_1', 'AC_EnsePosCangu_#NULL!', 'AC_SillaIncubadora_1', 'AC_SillaIncubadora_#NULL!',
    'V10B_1', 'V10B_2', 'V10B_3', 'V10B_4', 'V10B_5', 'V10B_6', 'V10B_7', 'V10B_8', 'V10B_#NULL!',
    'V10C_2', 'V10C_3', 'V10C_4', 'V10C_5', 'V10C_6', 'V10C_#NULL!', 'ANOCAT2ISS_2', 'ANOCAT2ISS_3',
    'ANOCAT2ISS_4', 'ANOCAT2ISS_5', 'ANOCAT2ISS_6', 'ANOCAT2ISS_7', 'ANOCAT2ISS_8', 'ANOCAT2ISS_9',
    'ANOCAT2ISS_#NULL!', 'educpadresimplificada_2', 'educpadresimplificada_3', 'educpadresimplificada_#NULL!',
    'ERN_sepsis_2', 'ERN_sepsis_3', 'ERN_sepsis_#NULL!', 'HD_C_GradoHIV_#NULL!', 'Sufrimientofetalcronico_1',
    'Sufrimientofetalcronico_#NULL!', 'tuvotransfusiones_1', 'tuvotransfusiones_#NULL!', 'toxemia_1.0',
    'ecocat_1', 'ecocat_2', 'ecocat_3', 'ecocat_#NULL!', 'apgarcat1_2', 'apgarcat1_3', 'apgarcat1_4',
    'apgarcat1_#NULL!', 'apgarcat5_2', 'apgarcat5_3', 'apgarcat5_4', 'apgarcat5_#NULL!',
    'ninosmenosde5anos_1', 'ninosmenosde5anos_#NULL!', 'UCI_1', 'UCI_#NULL!', 'ALIMENTAPARENTERAL_1',
    'ALIMENTAPARENTERAL_#NULL!', 'INFECCIONOSOCOMIAL_1', 'INFECCIONOSOCOMIAL_#NULL!', 'BPN_1.0',
    'Nearterm_1', 'Nearterm_#NULL!', 'controlcat_1', 'controlcat_2', 'controlcat_3', 'controlcat_#NULL!',
    'pdpcat_1', 'pdpcat_2', 'pdpcat_#NULL!', 'primipara_1', 'primipara_#NULL!', 'trimestre_1',
    'trimestre_2', 'trimestre_3', 'trimestre_#NULL!', 'anemiamadre_1', 'anemiamadre_#NULL!',
    'HD_Infecciones_1', 'HD_Infecciones_#NULL!', 'recibioantibioticos_1', 'recibioantibioticos_#NULL!',
    'AROCAT02_1', 'AROCAT02_2', 'AROCAT02_#NULL!', 'tipoventilacion_1', 'tipoventilacion_2',
    'tipoventilacion_3', 'tipoventilacion_#NULL!'
]

# Remoción de variables de identificación que no son predictoras
variables_predictoras = [v for v in variables_candidatas if not v.startswith('Iden_') and not v.startswith('VAR')]
variables_predictoras.append('Idenfinal') # Mantenemos Idenfinal si es el índice

# Asegúrate de que todas estas columnas existen en tu DataFrame 'df'
X = df[variables_predictoras]
Y = df['Riesgo_Neurocognitivo_Anual']

# División de los datos
X_train, X_test, Y_train, Y_test = train_test_split(
    X.drop(['Idenfinal'], axis=1), # Quitamos la variable de identificación para el entrenamiento
    Y,
    test_size=0.2,
    random_state=42,
    stratify=Y
)

print(f"Dimensiones de X_train: {X_train.shape}")
print(f"Dimensiones de X_test: {X_test.shape}")
print(f"Proporción de Riesgo (1) en Y_train: {Y_train.mean():.4f}")
print(f"Proporción de Riesgo (1) en Y_test: {Y_test.mean():.4f}")

Dimensiones de X_train: (51840, 540)
Dimensiones de X_test: (12961, 540)
Proporción de Riesgo (1) en Y_train: 0.0495
Proporción de Riesgo (1) en Y_test: 0.0495


## Entrenando el modelo Random Forest

In [39]:
# Variables que suelen ser de tipo datetime (o que causaron el error, ampliando la lista):
COLUMNAS_FECHA_A_EXCLUIR = [
    'Iden_FechaParto',
    'HD_FechaEntrada',
    'HD_FechaSalida',
    'ERN_FUM',
    'Iden_Codigo',
    'Iden_Sede'
]

# Paso 1: Reemplazar el string de error '#NULL!' por 0
VALOR_NULO_STRING = '#NULL!'
X_train_clean = X_train.replace(VALOR_NULO_STRING, 0)
X_test_clean = X_test.replace(VALOR_NULO_STRING, 0)

# Paso 2: Identificar y excluir columnas de forma dinámica
non_numeric_cols = X_train_clean.select_dtypes(include=['object', 'datetime']).columns.tolist()
cols_to_drop_final = list(set(non_numeric_cols + COLUMNAS_FECHA_A_EXCLUIR + ['Idenfinal']))
cols_to_drop_final = [col for col in cols_to_drop_final if col in X_train_clean.columns]

print(f"Columnas no numéricas excluidas: {cols_to_drop_final}")

X_train_clean = X_train_clean.drop(columns=cols_to_drop_final, errors='ignore')
X_test_clean = X_test_clean.drop(columns=cols_to_drop_final, errors='ignore')

# Paso 3: Intentar convertir el resto de las columnas a float
print("Convirtiendo el DataFrame restante a tipo float...")
X_train_clean = X_train_clean.astype(float)
X_test_clean = X_test_clean.astype(float)
print(f"Dimensiones finales de X_train (limpio): {X_train_clean.shape}")
print("Corrección completada. Reentrenando el modelo...")

# ENTRENAMIENTO DEL MODELO

# 1. Inicializar el modelo
modelo_rf = RandomForestClassifier(
    n_estimators=500,
    max_depth=10,
    min_samples_leaf=5,
    random_state=42,
    class_weight='balanced'
)

# 2. Entrenar el modelo con los datos LIMPIOS
print("\n--- Entrenando el modelo Random Forest ---")
modelo_rf.fit(X_train_clean, Y_train)
print("¡Entrenamiento completado!")

# EVALUACIÓN DEL RENDIMIENTO

# 1. Realizar predicciones sobre el conjunto de prueba limpio
Y_pred = modelo_rf.predict(X_test_clean)
Y_proba = modelo_rf.predict_proba(X_test_clean)[:, 1]

# 2. Imprimir Métricas de Rendimiento
print("\n---  Reporte de Clasificación en Conjunto de Prueba ---")
print(classification_report(Y_test, Y_pred))

print("\n---  Matriz de Confusión ---")
print(confusion_matrix(Y_test, Y_pred))

# 3. Métrica clave: Área bajo la Curva ROC (AUC-ROC)
auc_score = roc_auc_score(Y_test, Y_proba)
print(f"\n---  AUC-ROC Score: {auc_score:.4f} ---")

Columnas no numéricas excluidas: ['ERN_FUM', 'V195', 'V387', 'HD_FechaSalida', 'V215', 'HD_FechaEntrada', 'V7a', 'V259', 'V7', 'V302']
Convirtiendo el DataFrame restante a tipo float...
Dimensiones finales de X_train (limpio): (51840, 530)
Corrección completada. Reentrenando el modelo...

--- Entrenando el modelo Random Forest ---
¡Entrenamiento completado!

---  Reporte de Clasificación en Conjunto de Prueba ---
              precision    recall  f1-score   support

           0       0.98      0.79      0.87     12320
           1       0.14      0.67      0.23       641

    accuracy                           0.78     12961
   macro avg       0.56      0.73      0.55     12961
weighted avg       0.94      0.78      0.84     12961


---  Matriz de Confusión ---
[[9693 2627]
 [ 213  428]]

---  AUC-ROC Score: 0.8357 ---


## Entrenamiento XGBoost

In [40]:
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score


# Calcular el ratio de desbalance para el parámetro scale_pos_weight
# (Asumimos el ratio que teníamos antes: 12320 Normal / 641 Riesgo)
ratio_desbalance = Y_train.value_counts()[0] / Y_train.value_counts()[1]
print(f"Ratio de desbalance de clase (0/1): {ratio_desbalance:.2f}")

# 1. Inicializar el modelo XGBoost
modelo_xgb = xgb.XGBClassifier(
    objective='binary:logistic',
    n_estimators=300,            # Número de estimadores (árboles)
    max_depth=5,                 # Profundidad media
    learning_rate=0.1,           # Tasa de aprendizaje
    scale_pos_weight=ratio_desbalance, # Ajuste CRUCIAL para el desbalance
    tree_method='hist',          # Acelera el entrenamiento
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42,
    n_jobs=-1                    # Usar todos los núcleos disponibles
)

# 2. Entrenar el modelo
print("\n--- ⏳ Entrenando el modelo XGBoost ---")
# Usamos los datos limpios de la corrección anterior (X_train_clean, Y_train)
modelo_xgb.fit(X_train_clean, Y_train)
print("--- ✅ Entrenamiento XGBoost Completado ---")

# 3. Evaluación del Rendimiento
Y_pred_xgb = modelo_xgb.predict(X_test_clean)
Y_proba_xgb = modelo_xgb.predict_proba(X_test_clean)[:, 1]

print("\n---  Reporte de Clasificación en Conjunto de Prueba (XGBoost) ---")
print(classification_report(Y_test, Y_pred_xgb))

print("\n---  Matriz de Confusión (XGBoost) ---")
print(confusion_matrix(Y_test, Y_pred_xgb))

auc_score_xgb = roc_auc_score(Y_test, Y_proba_xgb)
print(f"\n---  AUC-ROC Score (XGBoost): {auc_score_xgb:.4f} ---")

Ratio de desbalance de clase (0/1): 19.20

--- ⏳ Entrenando el modelo XGBoost ---


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [17:37:57] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


--- ✅ Entrenamiento XGBoost Completado ---

---  Reporte de Clasificación en Conjunto de Prueba (XGBoost) ---
              precision    recall  f1-score   support

           0       0.98      0.85      0.91     12320
           1       0.18      0.63      0.28       641

    accuracy                           0.84     12961
   macro avg       0.58      0.74      0.60     12961
weighted avg       0.94      0.84      0.88     12961


---  Matriz de Confusión (XGBoost) ---
[[10529  1791]
 [  238   403]]

---  AUC-ROC Score (XGBoost): 0.8438 ---


##Entrenando el modelo de Regresión Logística (Lasso)

In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score


# 1. Inicializar el modelo de Regresión Logística con Lasso
# C (inverso de la fuerza de regularización) se establece en 0.1 para forzar
# que Lasso elimine algunas variables poco importantes.
modelo_logreg = LogisticRegression(
    penalty='l1',
    solver='liblinear',
    C=0.1,  # Fuerte regularización
    class_weight='balanced', # Crucial para el desbalance
    random_state=42,
    max_iter=500
)

# 2. Entrenar el modelo
print("\n---  Entrenando el modelo de Regresión Logística (Lasso) ---")
# Usamos los mismos datos limpios (X_train_clean, Y_train)
modelo_logreg.fit(X_train_clean, Y_train)
print("---  Entrenamiento Regresión Logística Completado ---")

# 3. Evaluación del Rendimiento
Y_pred_logreg = modelo_logreg.predict(X_test_clean)
Y_proba_logreg = modelo_logreg.predict_proba(X_test_clean)[:, 1]

print("\n---  Reporte de Clasificación en Conjunto de Prueba (Reg. Logística) ---")
print(classification_report(Y_test, Y_pred_logreg))

print("\n---  Matriz de Confusión (Reg. Logística) ---")
print(confusion_matrix(Y_test, Y_pred_logreg))

auc_score_logreg = roc_auc_score(Y_test, Y_proba_logreg)
print(f"\n---  AUC-ROC Score (Reg. Logística): {auc_score_logreg:.4f} ---")





---  Entrenando el modelo de Regresión Logística (Lasso) ---
---  Entrenamiento Regresión Logística Completado ---

---  Reporte de Clasificación en Conjunto de Prueba (Reg. Logística) ---
              precision    recall  f1-score   support

           0       0.98      0.74      0.85     12320
           1       0.13      0.78      0.23       641

    accuracy                           0.74     12961
   macro avg       0.56      0.76      0.54     12961
weighted avg       0.94      0.74      0.82     12961


---  Matriz de Confusión (Reg. Logística) ---
[[9129 3191]
 [ 144  497]]

---  AUC-ROC Score (Reg. Logística): 0.8453 ---


##Undersampling 3:1

In [44]:
from imblearn.under_sampling import RandomUnderSampler


# --- 1. Definición del Ratio ---

# Contar las muestras de la clase minoritaria (Clase 1: Riesgo)
n_minority = Y_train.value_counts()[1] # 2566 muestras

# Estrategia 3:1 (Reducir la clase 0 a tres veces el tamaño de la clase 1)
sampling_strategy_3_1 = {
    0: n_minority * 3,
    1: n_minority
}

# --- 2. Inicialización y Aplicación del Undersampling ---
rus_3_1 = RandomUnderSampler(
    sampling_strategy=sampling_strategy_3_1,
    random_state=42
)

print("Aplicando Undersampling Aleatorio (Ratio 3:1) al conjunto de entrenamiento...")
# Usamos X_train_clean y Y_train, que son los datos originales limpios
X_train_resampled_rus, Y_train_resampled_rus = rus_3_1.fit_resample(X_train_clean, Y_train)
print("--- ✅ Aplicación Undersampling Completada ---")

# --- 3. Reporte de Dimensiones ---

print("\n--- 📊 Resultados del Muestreo (Undersampling 3:1) ---")
print(f"Dimensiones Originales de X_train: {X_train_clean.shape}")
print(f"Distribución de Clases Original: {Y_train.value_counts().tolist()}")
print(f"Dimensiones de X_train Despues de Undersampling: {X_train_resampled_rus.shape}")
print(f"Distribución de Clases Despues de Undersampling (3:1): {Y_train_resampled_rus.value_counts().tolist()}")

# Las variables X_train_resampled_rus y Y_train_resampled_rus ahora contienen
# el nuevo conjunto de datos de entrenamiento subrepresentado (3:1).

Aplicando Undersampling Aleatorio (Ratio 3:1) al conjunto de entrenamiento...
--- ✅ Aplicación Undersampling Completada ---

--- 📊 Resultados del Muestreo (Undersampling 3:1) ---
Dimensiones Originales de X_train: (51840, 530)
Distribución de Clases Original: [49274, 2566]
Dimensiones de X_train Despues de Undersampling: (10264, 530)
Distribución de Clases Despues de Undersampling (3:1): [7698, 2566]


##SMOTE 1:1

In [45]:
from imblearn.over_sampling import SMOTE

#X_train_resampled_rus y Y_train_resampled_rus son los resultados del Undersampling 3:1.

# --- 1. Inicialización de SMOTE ---
smote_hybrid = SMOTE(sampling_strategy='auto', random_state=42)

# --- 2. Aplicación de SMOTE ---

print("Aplicando SMOTE (Oversampling Sintético) al dataset Subrepresentado (Híbrido)...")
X_train_resampled_hybrid, Y_train_resampled_hybrid = smote_hybrid.fit_resample(X_train_resampled_rus, Y_train_resampled_rus)
print("--- Aplicación SMOTE (Híbrido) Completada ---")

# --- 3. Reporte de Dimensiones ---

print("\n--- 📊 Resultados del Muestreo Híbrido ---")
print(f"Dimensiones Iniciales (3:1 Undersampling): {X_train_resampled_rus.shape}")
print(f"Distribución de Clases Inicial (3:1): {Y_train_resampled_rus.value_counts().tolist()}")
print(f"Dimensiones de X_train Híbrido: {X_train_resampled_hybrid.shape}")
print(f"Distribución de Clases Híbrido (1:1): {Y_train_resampled_hybrid.value_counts().tolist()}")


Aplicando SMOTE (Oversampling Sintético) al dataset Subrepresentado (Híbrido)...
--- Aplicación SMOTE (Híbrido) Completada ---

--- 📊 Resultados del Muestreo Híbrido ---
Dimensiones Iniciales (3:1 Undersampling): (10264, 530)
Distribución de Clases Inicial (3:1): [7698, 2566]
Dimensiones de X_train Híbrido: (15396, 530)
Distribución de Clases Híbrido (1:1): [7698, 7698]


##Entrenando XGBoost con Muestreo Híbrido

In [46]:
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import pandas as pd
import numpy as np

# NOTA: X_train_resampled_hybrid y Y_train_resampled_hybrid son los datos del muestreo híbrido 1:1.

# 1. Inicializar el modelo XGBoost
# Usaremos los mismos parámetros que nos dieron el mejor resultado.
modelo_xgb_hybrid_final = xgb.XGBClassifier(
    objective='binary:logistic',
    n_estimators=300,
    max_depth=5,
    learning_rate=0.1,
    scale_pos_weight=1, # Desactivado/Neutro porque los datos ya están balanceados (1:1).
    tree_method='hist',
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42,
    n_jobs=-1
)

# 2. Entrenar el modelo con los datos balanceados híbridos
print("\n--- Entrenando XGBoost con Muestreo Híbrido ---")
modelo_xgb_hybrid_final.fit(X_train_resampled_hybrid, Y_train_resampled_hybrid)
print("--- Re-entrenamiento Completado ---")

# 3. Evaluación del Rendimiento en el conjunto de PRUEBA original (X_test_clean)
Y_pred_hybrid_final = modelo_xgb_hybrid_final.predict(X_test_clean)
Y_proba_hybrid_final = modelo_xgb_hybrid_final.predict_proba(X_test_clean)[:, 1]

print("\n---  Reporte de Clasificación (XGBoost Híbrido) ---")
print(classification_report(Y_test, Y_pred_hybrid_final))

print("\n--- Matriz de Confusión (XGBoost Híbrido) ---")
print(confusion_matrix(Y_test, Y_pred_hybrid_final))

auc_score_hybrid_final = roc_auc_score(Y_test, Y_proba_hybrid_final)
print(f"\n---  AUC-ROC Score (XGBoost Híbrido): {auc_score_hybrid_final:.4f} ---")


--- Entrenando XGBoost con Muestreo Híbrido ---


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [17:51:09] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


--- Re-entrenamiento Completado ---

---  Reporte de Clasificación (XGBoost Híbrido) ---
              precision    recall  f1-score   support

           0       0.97      0.91      0.94     12320
           1       0.23      0.51      0.31       641

    accuracy                           0.89     12961
   macro avg       0.60      0.71      0.63     12961
weighted avg       0.94      0.89      0.91     12961


--- Matriz de Confusión (XGBoost Híbrido) ---
[[11205  1115]
 [  313   328]]

---  AUC-ROC Score (XGBoost Híbrido): 0.8481 ---


##Buscando el Umbral Óptimo para F1-Score

In [47]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score

# NOTA: Debemos usar Y_proba_hybrid_final (probabilidades de predicción del modelo final)
# y Y_test (las etiquetas reales).

# Obtener las probabilidades de riesgo (Clase 1)
Y_proba = modelo_xgb_hybrid_final.predict_proba(X_test_clean)[:, 1]

# Definir un rango de umbrales para probar (de 0.05 a 0.95)
thresholds = np.arange(0.05, 0.95, 0.01)

results = []

print("--- Buscando el Umbral Óptimo para F1-Score ---")
for t in thresholds:
    # Convertir probabilidades a predicciones binarias con el umbral 't'
    Y_pred_new = (Y_proba >= t).astype(int)

    # Calcular las métricas
    f1 = f1_score(Y_test, Y_pred_new)
    precision = precision_score(Y_test, Y_pred_new)
    recall = recall_score(Y_test, Y_pred_new)

    results.append({
        'Threshold': t,
        'F1-Score': f1,
        'Precision': precision,
        'Recall': recall
    })

results_df = pd.DataFrame(results)

# Encontrar el umbral que maximiza el F1-Score
best_threshold_row = results_df.loc[results_df['F1-Score'].idxmax()]

print("\n--- 🏆 Resultado Óptimo Encontrado: ---")
print(f"Mejor Umbral: {best_threshold_row['Threshold']:.2f}")
print(f"F1-Score Máximo: {best_threshold_row['F1-Score']:.4f}")
print(f"Precision en este umbral: {best_threshold_row['Precision']:.4f}")
print(f"Recall en este umbral: {best_threshold_row['Recall']:.4f}")

--- Buscando el Umbral Óptimo para F1-Score ---

--- 🏆 Resultado Óptimo Encontrado: ---
Mejor Umbral: 0.56
F1-Score Máximo: 0.3228
Precision en este umbral: 0.2555
Recall en este umbral: 0.4384


##XGBoost con scale_pos_weight = 10

In [51]:
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score


# 1. Definir el nuevo peso de clase (scale_pos_weight = 10)
# Esto es un castigo más suave que el desbalance real (19.2)
nuevo_peso = 10

# 2. Inicializar el modelo XGBoost
modelo_xgb_peso = xgb.XGBClassifier(
    objective='binary:logistic',
    n_estimators=300,
    max_depth=5,
    learning_rate=0.1,
    scale_pos_weight=nuevo_peso,
    tree_method='hist',
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42,
    n_jobs=-1
)

# 3. Entrenar el modelo con los datos limpios originales (X_train_clean, Y_train)
print("\n--- ⏳ Re-entrenando XGBoost con scale_pos_weight = 10 ---")
modelo_xgb_peso.fit(X_train_clean, Y_train)
print("--- ✅ Re-entrenamiento Completado ---")

# 4. Evaluación del Rendimiento
Y_pred_peso = modelo_xgb_peso.predict(X_test_clean)
Y_proba_peso = modelo_xgb_peso.predict_proba(X_test_clean)[:, 1]

print("\n---  Reporte de Clasificación (XGBoost con Peso=10) ---")
print(classification_report(Y_test, Y_pred_peso))

print("\n---  Matriz de Confusión (XGBoost con Peso=10) ---")
print(confusion_matrix(Y_test, Y_pred_peso))

auc_score_peso = roc_auc_score(Y_test, Y_proba_peso)
print(f"\n---  AUC-ROC Score (XGBoost con Peso=10): {auc_score_peso:.4f} ---")


--- ⏳ Re-entrenando XGBoost con scale_pos_weight = 10 ---


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [17:59:44] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


--- ✅ Re-entrenamiento Completado ---

---  Reporte de Clasificación (XGBoost con Peso=10) ---
              precision    recall  f1-score   support

           0       0.97      0.92      0.94     12320
           1       0.23      0.49      0.32       641

    accuracy                           0.89     12961
   macro avg       0.60      0.70      0.63     12961
weighted avg       0.94      0.89      0.91     12961


---  Matriz de Confusión (XGBoost con Peso=10) ---
[[11284  1036]
 [  325   316]]

---  AUC-ROC Score (XGBoost con Peso=10): 0.8490 ---


In [49]:
# Usamos el modelo final entrenado: modelo_xgb_peso
# X_train_clean contiene los nombres de las 530 columnas originales.

import pandas as pd
import numpy as np

# Extraer la importancia de las variables (Feature Importance)
feature_importances_peso = pd.Series(
    modelo_xgb_peso.feature_importances_,
    index=X_train_clean.columns
).sort_values(ascending=False)

# 1. Variables con importancia > 0
important_features_peso = feature_importances_peso[feature_importances_peso > 0]

print("\n--- 🧠 Importancia de Variables (XGBoost con Peso=10) ---")
print(f"Total de variables utilizadas por el modelo: {len(important_features_peso)} (de {X_train_clean.shape[1]})")

# 2. Imprimir las 20 variables más importantes
print("\n--- 🔝 Top 20 Variables más Relevantes (Factores de Riesgo/Protección) ---")
print(important_features_peso.head(20).to_string())


--- 🧠 Importancia de Variables (XGBoost con Peso=10) ---
Total de variables utilizadas por el modelo: 427 (de 530)

--- 🔝 Top 20 Variables más Relevantes (Factores de Riesgo/Protección) ---
V10A                              0.077896
HD_C_Ictericia_#NULL!             0.054849
CSP_TipoVivienda_#NULL!           0.039702
BMImadrecat_Cod                   0.035616
HD_C_OxigenoDependencia_#NULL!    0.033283
AC_LugarAC_#NULL!                 0.022390
V10C_#NULL!                       0.022133
CSP_NutricionFam_2                0.015871
periodosanalisis_#NULL!           0.012222
PA_AtendioParto_#NULL!            0.011522
V10B_#NULL!                       0.010937
periodosanalisis_4                0.009287
CP_TP_Sifilis_#NULL!              0.009095
Sistemadeaseguramiento_3.0        0.007509
HD_TotalDiasHospital              0.005893
CP_TP_HIV_#NULL!                  0.005857
PA_Monitoreo_1                    0.005817
PA_NumDosisCorticoides_#NULL!     0.005428
gestacat                          0